In [0]:
# Run once then comment out
%pip install confluent-kafka

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import confluent_kafka as kafka
import json
from datetime import datetime, timedelta
from pyspark.sql.functions import col

In [0]:
props = {
    # our class settings:
    'sasl.username':'UGZQZXBMU4ITW4TE',
    'sasl.password': 'vWTRzZdjZxnZu9NTpgBctijiq6lxNmMMcZ3tZejvVftLTvEjS2eTb1OSeKPVww4F',
    'bootstrap.servers':'pkc-56d1g.eastus.azure.confluent.cloud:9092',
    # boilerplate settings
    'security.protocol':'SASL_SSL',
    'sasl.mechanisms':'PLAIN',
    'session.timeout.ms':45000,
    'auto.offset.reset':'earliest',
    'group.id':'python-group-25',
}

consumer = kafka.Consumer(props)
consumer.subscribe(['chi-crime'])

In [0]:
timeout_in_sec = 30
try:
    messages = []
    start_time = datetime.now()
    while True:
        msg = consumer.poll(5.0)
        if msg is not None and msg.error() is None:
            messages.append(msg)
        if datetime.now() >= start_time + timedelta(seconds=timeout_in_sec):
            raise Exception('Timeout')
except Exception as e:
    print(f'Exception: {str(e)[:100]}')
finally:
    consumer.close()
    print(f'{len(messages)} messages have been received!')

Exception: Timeout
105 messages have been received!


In [0]:
df_cols = list(json.loads(messages[0].value().decode('utf-8')).keys())
df_data = []

for i in messages:
    data = list(json.loads(i.value().decode('utf-8')).values())
    df_data.append(data)
    
df = spark.createDataFrame(df_data, df_cols)
# df.display()

In [0]:
df = df.drop('y_coordinate', 'x_coordinate')
df = df.withColumn('id', df.id.cast('int'))
df = df.withColumn('beat', df.beat.cast('int'))
df = df.withColumn('district', df.district.cast('int'))
df = df.withColumn('date', df.date.cast('timestamp'))
df = df.withColumn('latitude', df.latitude.cast('float'))
df = df.withColumn('longitude', df.longitude.cast('float'))

df = df.dropDuplicates()

print(df.count())

105


In [0]:
from pyspark.sql.functions import col, min, max

server = 'cohort40-sql-2.database.windows.net'
database = 'test-ck'
user = 'testuser25'
password = 'T3STPASSWORD!!'
table = 'dbo.crime'

max_id = df.agg(max('ID')).collect()[0][0]
min_id = df.agg(min('ID')).collect()[0][0]

# read in sql database
query = f"(SELECT id FROM {table} WHERE ID BETWEEN {min_id} AND {max_id}) as t"
jdbcDF = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", query) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

df = df.join(jdbcDF, 'ID', 'leftanti')

In [0]:
# add new rows to SQL database
df.write.format('jdbc').option(
    'url', f'jdbc:sqlserver://{server}:1433;databaseName={database};'
    ) \
    .mode('append') \
    .option('dbtable', table) \
    .option('user', user) \
    .option('password', password) \
    .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
    .save()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2470460636152967>:2
      1 # add new rows to SQL database
----> 2 df.write.format('jdbc').option(
      3     'url', f'jdbc:sqlserver://{server}:1433;databaseName={database};'
      4     ) \
      5     .mode('append') \
      6     .option('dbtable', table) \
      7     .option('user', user) \
      8     .option('password', password) \
      9     .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
     10     .save()

NameError: name 'df' is not defined